In [1]:
#Depedencies
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
api_key = config.coin_api_key

In [1]:
#Code to authenticate notebook 
import pandas as pd
from azure.datalake.store import core, lib, multithread
token = lib.auth()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EPDMCYMU7 to authenticate.


In [8]:
import pandas as pd
from azure.datalake.store import core, lib, multithread
token = lib.auth()
# Create an ADLS File System Client. The store_name is the name of your ADLS account
adl = core.AzureDLFileSystem(token = token, store_name='coinstoragenjr')

adl.ls('')

# Read a file into pandas dataframe
with adl.open('sampledata2.csv', 'rb') as f:
    df = pd.read_csv(f) 

# Show the dataframe
df

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F4XTUY5GP to authenticate.


DatalakeRESTException: HTTP error: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='coinstoragenjr.azuredatalakestore.net', port=443): Max retries exceeded with url: /webhdfs/v1/?OP=LISTSTATUS&api-version=2018-09-01&listSize=4000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A3A864A160>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"))

In [9]:
adl = core.AzureDLFileSystem(token = token, store_name='trial')

adl.ls('')

PermissionError: .

In [6]:
adlsFileSystemClient = core.AzureDLFileSystem(token, store_name='coinstoragenjr')
adl = core.AzureDLFileSystem(token, store_name='coinstoragenjr')
adl.ls('')

DatalakeRESTException: HTTP error: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='coinstoragenjr.azuredatalakestore.net', port=443): Max retries exceeded with url: /webhdfs/v1/?OP=LISTSTATUS&api-version=2018-09-01&listSize=4000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A3A863F520>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"))

In [2]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
    'listing_status':'active',
    'limit': 100,
    'sort': 'cmc_rank'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': api_key
}

session = Session()
session.headers.update(headers)
try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [3]:
#get the current datetime to add to the file
from datetime import datetime
now = datetime.now()
injest_date = now.strftime("%m/%d/%Y %H:%M:%S")

In [4]:
#grabs section of dictionary 
cmc_data = data['data']
#puts the dict into a DF, now need to expand nested platform column
cmc_df = pd.DataFrame.from_dict(cmc_data, orient='columns')
#expanded Datafram
full_df = pd.concat([cmc_df.drop(['platform'], axis= 1), cmc_df['platform'].apply(pd.Series)], axis= 1 )
#add column file name
full_df["File_Name"] = "Coin_Ranking_Dim"
#Add column for injested at 
full_df["injest_datetime"] = injest_date
full_df.head()

#turn dataframe to CSV
full_df_CSV = full_df.to_csv()


 - write DF to parquet format
 `trial_write_to_parquet = test.to_parquet(headers = 'none')`
 - write DF to CSV format
`dfasCSV = test.to_csv()`
 - write DF to string format
`dfAsString = test.to_string(header=False, index=False)`


In [5]:
#this works, needed to send it as text !!
#it works too, had to save it as a CSV
from azure.storage.filedatalake import DataLakeServiceClient
# install the following package 
# pip install azure-storage-file-datalake 
# Get the below details from your storage account
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = config.container_name
directory_name = config.directory_name

service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", storage_account_name), credential=storage_account_key)

file_system_client = service_client.get_file_system_client(file_system=container_name)
dir_client = file_system_client.get_directory_client(directory_name)
dir_client.create_directory()
#set data to appropriate dataframe
data = full_df_CSV 


#make sure to change file type as well
file_client = dir_client.create_file("sampledata2.csv")
file_client.append_data(data, 0, len(data))
file_client.flush_data(len(data))

{'date': datetime.datetime(2022, 3, 29, 22, 45, 18, tzinfo=datetime.timezone.utc),
 'etag': '"0x8DA11D5CCE697B4"',
 'last_modified': datetime.datetime(2022, 3, 29, 22, 45, 18, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': 'e89e31bd-afb1-11ec-b623-74d83ee76d90',
 'request_id': '1a01c476-101f-0058-57be-43e979000000',
 'version': '2020-10-02'}

### -----------------------------------------------------------------------------------------------------------------------------------------

In [6]:
#Read CSV file from ADLS,
#clean up file and then write it back to storage
service = DataLakeServiceClient.from_connection_string(conn_str= config.march29connectString)

file = DataLakeFileClient.from_connection_string("config.march29connectString",
                                                 file_system_name="myfilesystem", file_path="myfile")

with open("./BlockDestination.txt", "wb") as my_file:
    download = file.download_file()
    download.readinto(my_file)

AttributeError: module 'config' has no attribute 'march29connectString'